In [1]:
import pandas as pd
import os,sys

In [2]:
sys.path.append('../../scripts/cddd/cddd/')
sys.path.append('../../scripts/default_model')
sys.path.append('../../scripts/')
sys.path.append('../../plot_styles/')
sys.path.append('../../data/')


In [3]:
os.path.abspath("../..")

'/home/suman/Shaheerah/mtan_admet'

In [4]:
import pandas as pd
from inference import InferenceModel
from preprocessing import preprocess_smiles
import numpy as np

In [5]:
admet_df = pd.read_csv('../../data/admet.csv', error_bad_lines=False)
admet_df.columns
# Get non-NaN counts per column
non_nan_counts = admet_df.notna().sum()

# Print with column name and count
for col, count in non_nan_counts.items():
    print(f"{col}: {count}")


HIA: 734
OB: 1055
p-gp inhibitor: 2077
p-gp substrates: 888
BBB: 2320
Respiratory toxicity: 1399
Hepatotoxicity: 1313
half-life: 1323
CL: 1338
Cardiotoxicity1: 1565
Cardiotoxicity10: 1565
Cardiotoxicity30: 1565
Cardiotoxicity5: 1565
CYP1A2: 9748
CYP2C19: 10598
CYP2C9: 9906
CYP2D6: 10682
CYP3A4: 11369
Acute oral toxicity (LD50): 7313
IGC50: 1787
ESOL: 1128
logD: 1130
Caco-2 permeability: 4463
PPB: 1830
group: 48390
smiles: 48390


In [6]:
'''
preprocesses a SMILES string such that it is in the same format as
the translation model was trained on. It removes salts and stereochemistry from the
SMILES sequnce. If the sequnce correspond to an inorganic molecule or cannot be
interpreted by RDKit nan is returned.
'''

admet_df["smiles_preprocessed"] = admet_df.smiles.map(preprocess_smiles)
smiles_list = admet_df["smiles_preprocessed"].to_numpy()
admet_df.shape

(48390, 27)

In [7]:
import tensorflow as tf

# Set TensorFlow logging level to ERROR to suppress INFO and WARNING messages
tf.get_logger().setLevel('ERROR')

inference_model = InferenceModel(model_dir="../../scripts/default_model")

In [8]:
# Define a list to hold invalid entries
invalid_entries = []

# Process and filter smiles_list, while collecting invalid entries
smiles_list_cleaned = []
for i,smiles in enumerate(smiles_list):
    if isinstance(smiles, str):
        smiles_list_cleaned.append(smiles)
    else:
        invalid_entries.append(i)

# Print the invalid entries
if invalid_entries:
    print("Invalid entries found in smiles_list:")
    for entry in invalid_entries:
        print(f"Invalid entry: {entry} (Type: {type(entry)})")

admet_cleaned_df = admet_df.drop(invalid_entries).reset_index(drop=True)
# Replace smiles_list with the cleaned version
smiles_list = admet_cleaned_df["smiles_preprocessed"].to_numpy()


Invalid entries found in smiles_list:
Invalid entry: 22 (Type: <class 'int'>)
Invalid entry: 29 (Type: <class 'int'>)
Invalid entry: 50 (Type: <class 'int'>)
Invalid entry: 79 (Type: <class 'int'>)
Invalid entry: 115 (Type: <class 'int'>)
Invalid entry: 144 (Type: <class 'int'>)
Invalid entry: 186 (Type: <class 'int'>)
Invalid entry: 200 (Type: <class 'int'>)
Invalid entry: 207 (Type: <class 'int'>)
Invalid entry: 228 (Type: <class 'int'>)
Invalid entry: 269 (Type: <class 'int'>)
Invalid entry: 294 (Type: <class 'int'>)
Invalid entry: 306 (Type: <class 'int'>)
Invalid entry: 357 (Type: <class 'int'>)
Invalid entry: 370 (Type: <class 'int'>)
Invalid entry: 376 (Type: <class 'int'>)
Invalid entry: 401 (Type: <class 'int'>)
Invalid entry: 404 (Type: <class 'int'>)
Invalid entry: 421 (Type: <class 'int'>)
Invalid entry: 437 (Type: <class 'int'>)
Invalid entry: 481 (Type: <class 'int'>)
Invalid entry: 504 (Type: <class 'int'>)
Invalid entry: 545 (Type: <class 'int'>)
Invalid entry: 549 (Typ

Invalid entry: 34362 (Type: <class 'int'>)
Invalid entry: 34377 (Type: <class 'int'>)
Invalid entry: 34379 (Type: <class 'int'>)
Invalid entry: 34452 (Type: <class 'int'>)
Invalid entry: 34454 (Type: <class 'int'>)
Invalid entry: 34494 (Type: <class 'int'>)
Invalid entry: 34514 (Type: <class 'int'>)
Invalid entry: 34548 (Type: <class 'int'>)
Invalid entry: 34564 (Type: <class 'int'>)
Invalid entry: 34565 (Type: <class 'int'>)
Invalid entry: 34567 (Type: <class 'int'>)
Invalid entry: 34597 (Type: <class 'int'>)
Invalid entry: 34598 (Type: <class 'int'>)
Invalid entry: 34721 (Type: <class 'int'>)
Invalid entry: 34779 (Type: <class 'int'>)
Invalid entry: 34781 (Type: <class 'int'>)
Invalid entry: 34789 (Type: <class 'int'>)
Invalid entry: 34810 (Type: <class 'int'>)
Invalid entry: 34826 (Type: <class 'int'>)
Invalid entry: 34868 (Type: <class 'int'>)
Invalid entry: 34907 (Type: <class 'int'>)
Invalid entry: 34909 (Type: <class 'int'>)
Invalid entry: 34943 (Type: <class 'int'>)
Invalid ent

In [9]:
print(len(invalid_entries))

3406


In [10]:
admet_cleaned_df.columns

Index(['HIA', 'OB', 'p-gp inhibitor', 'p-gp substrates', 'BBB',
       'Respiratory toxicity', 'Hepatotoxicity', 'half-life', 'CL',
       'Cardiotoxicity1', 'Cardiotoxicity10', 'Cardiotoxicity30',
       'Cardiotoxicity5', 'CYP1A2', 'CYP2C19', 'CYP2C9', 'CYP2D6', 'CYP3A4',
       'Acute oral toxicity (LD50)', 'IGC50', 'ESOL', 'logD',
       'Caco-2 permeability', 'PPB', 'group', 'smiles', 'smiles_preprocessed'],
      dtype='object')

In [11]:
# Filter out non-string elements from smiles_list
smiles_list = [str(smiles) for smiles in smiles_list if isinstance(smiles, str)]


In [12]:
print(len(smiles_list))
print(admet_cleaned_df.shape)

44984
(44984, 27)


In [13]:
import numpy as np
from joblib import Parallel, delayed
from tqdm import tqdm  

# Function to process each SMILES string and get its embedding
def process_smiles(smiles,index):
    """
    Process a SMILES string to get its embedding.

    Args:
        smiles (str): The SMILES string to process.
        index (int): The index of the SMILES string in the list (for error reporting).

    Returns:
        np.ndarray or None: The flattened embedding if successful, or None if an error occurs.
    """
    #print(index)
    try:
        # Compute the embedding for the given SMILES string
        embedding = inference_model.seq_to_emb(smiles)

        # Ensure embedding has the expected shape
        if embedding.shape == (1, 512):
            #print(smiles)
            return embedding.flatten()
        else:
            print(f"Unexpected embedding shape for SMILES {index}: {embedding.shape}")
            return None
    except Exception as e:
        print(f"Error processing SMILES {index}: {e}")
        return None





In [14]:
# Use joblib's Parallel and delayed for parallel processing
with tqdm(total=len(smiles_list), desc="Processing SMILES") as pbar:
    def update_progress(smiles, index):
        result = process_smiles(smiles, index)
        pbar.update(1)  
        return result

    results = Parallel(n_jobs=-1, verbose=0,backend = 'threading')(
        delayed(update_progress)(smiles_list[i], i) for i in range(len(smiles_list))# cant be pickeled so use threading
    )

Processing SMILES: 100%|██████████| 44984/44984 [48:31<00:00, 15.45it/s]  


In [15]:
# Initialize the array to store embeddings
embedding_dim = 512
smiles_embeddings = np.zeros((len(smiles_list), embedding_dim))
# Fill the smiles_embeddings array with valid results
for i, result in enumerate(results):
    if result is not None:
        smiles_embeddings[i, :] = result

In [16]:
smiles_embeddings.shape

(44984, 512)

In [17]:
admet_cleaned_df['embeddings'] = list(smiles_embeddings)

In [18]:
admet_cleaned_df.columns

Index(['HIA', 'OB', 'p-gp inhibitor', 'p-gp substrates', 'BBB',
       'Respiratory toxicity', 'Hepatotoxicity', 'half-life', 'CL',
       'Cardiotoxicity1', 'Cardiotoxicity10', 'Cardiotoxicity30',
       'Cardiotoxicity5', 'CYP1A2', 'CYP2C19', 'CYP2C9', 'CYP2D6', 'CYP3A4',
       'Acute oral toxicity (LD50)', 'IGC50', 'ESOL', 'logD',
       'Caco-2 permeability', 'PPB', 'group', 'smiles', 'smiles_preprocessed',
       'embeddings'],
      dtype='object')

In [19]:
admet_cleaned_df.to_csv("../../data/admet_embeddings_clean.csv")

In [20]:
embeddings_prop_df = admet_cleaned_df.drop(columns=['group','smiles_preprocessed','smiles'])

In [21]:
embeddings_prop_df.columns
# Get non-NaN counts per column
non_nan_counts = embeddings_prop_df.notna().sum()

# Print with column name and count
for col, count in non_nan_counts.items():
    print(f"{col}: {count}")


HIA: 706
OB: 975
p-gp inhibitor: 1831
p-gp substrates: 732
BBB: 2186
Respiratory toxicity: 1351
Hepatotoxicity: 1212
half-life: 1109
CL: 1123
Cardiotoxicity1: 1491
Cardiotoxicity10: 1491
Cardiotoxicity30: 1491
Cardiotoxicity5: 1491
CYP1A2: 9373
CYP2C19: 10189
CYP2C9: 9512
CYP2D6: 10251
CYP3A4: 10845
Acute oral toxicity (LD50): 7009
IGC50: 1784
ESOL: 1081
logD: 1035
Caco-2 permeability: 3856
PPB: 1609
embeddings: 44984


In [22]:
features = embeddings_prop_df.to_numpy()[:,:-1]  # Exclude the last column (embeddings)
embeddings = np.array(embeddings_prop_df['embeddings'].to_numpy())  # Convert embeddings column to numpy array

In [23]:
data = np.hstack([smiles_embeddings,features])

In [24]:
data.shape

(44984, 536)

In [25]:
np.savetxt("../../data/embeddings_features.dat",data, delimiter= " ")